1.Read the data into an RDD

In [15]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

rdd = sc.textFile("EPL.csv")
header = rdd.first()
rdd = rdd.filter(lambda row: row != header and len(row.split(",")) == 6).map(lambda row: row.split(","))


2 .Which season has the highest number of goals

In [11]:
season_goals = rdd.map(lambda x: (x[5], int(float(x[2])) + int(float(x[3])))).reduceByKey(lambda a, b: a + b)
highest_season = season_goals.max(key=lambda x: x[1])
print("Season with highest number of goals:", highest_season)

Season with highest number of goals: ('2011-2012', 1066)


3 .Which season has the lowest number of goals

In [12]:

lowest_season = season_goals.min(key=lambda x: x[1])
print("Season with lowest number of goals:", lowest_season)

Season with lowest number of goals: ('2006-2007', 931)


4 .Which team has the highest average goals per season

In [13]:
team_goals = rdd.flatMap(lambda x: [
    ((x[0], x[5]), int(float(x[2]))),
    ((x[1], x[5]), int(float(x[3])))
])
team_season_goals = team_goals.reduceByKey(lambda a, b: a + b)
team_total = team_season_goals.map(lambda x: (x[0][0], (x[1], {x[0][1]}))) \
                              .reduceByKey(lambda a, b: (a[0] + b[0], a[1].union(b[1])))
team_avg = team_total.map(lambda x: (x[0], x[1][0] / len(x[1][1])))
best_team = team_avg.max(key=lambda x: x[1])
print("Team with highest average goals per season:", best_team)

Team with highest average goals per season: ('Manchester United', 72.25)


5 .Probabilities of Manchester United results

In [14]:
mu_matches = rdd.filter(lambda x: x[0] == "Manchester United" or x[1] == "Manchester United")
total_matches = mu_matches.count()
mu_wins = mu_matches.filter(lambda x: (x[0] == "Manchester United" and x[4] == "H") or (x[1] == "Manchester United" and x[4] == "A")).count()
mu_losses = mu_matches.filter(lambda x: (x[0] == "Manchester United" and x[4] == "A") or (x[1] == "Manchester United" and x[4] == "H")).count()
mu_draws = mu_matches.filter(lambda x: x[4] == "D").count()
p_win = mu_wins / total_matches if total_matches > 0 else 0
p_loss = mu_losses / total_matches if total_matches > 0 else 0
p_draw = mu_draws / total_matches if total_matches > 0 else 0

print("Probability that Manchester United Wins:", round(p_win, 2))
print("Probability that Manchester United Loses:", round(p_loss, 2))
print("Probability that Manchester United Draws:", round(p_draw, 2))


Probability that Manchester United Wins: 0.64
Probability that Manchester United Loses: 0.17
Probability that Manchester United Draws: 0.2
